In [1]:
import syft as sy
sy.requires(">=0.8,<0.8.1")

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/yunhs


✅ The installed version of syft==0.8.0 matches the requirement >=0.8 and the requirement <0.8.1


In [2]:
node = sy.orchestra.launch(name="test-domain-1", port=8080, dev_mode=True)

Starting test-domain-1 server on 0.0.0.0:8080

SQLite Store Path:
!open file:///var/folders/4s/vsgdjz495453yymsd09nx80c0000gn/T/7bca415d13ed1ec841f0d0aede098dbb.sqlite

> Domain: test-domain-1 - 7bca415d13ed1ec841f0d0aede098dbb - NodeType.DOMAIN

Services:
ActionService
DataSubjectMemberService
DataSubjectService
DatasetService
MessageService
MetadataService
NetworkService
PolicyService
ProjectService
RequestService
UserCodeService
UserService


INFO:     Started server process [2758]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)


INFO:     127.0.0.1:49727 - "GET /api/v1/new/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:49727 - "POST /api/v1/new/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:49727 - "GET /api/v1/new/api?verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f HTTP/1.1" 200 OK
INFO:     127.0.0.1:49732 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:49736 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:49739 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:49742 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:49745 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:49748 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:49752 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:49755 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:49758 - "POST /api/v1/new/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:49761 - "POST /api/v1/new/api_call HTTP/1.1" 200 O

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [2758]


In [3]:
domain_client = node.login(email="info@openmined.org", password="changethis")

In [4]:
domain_client.notifications

,type,id,subject,status,created_at,linked_obj
0,syft.service.message.messages.Message,8cf4d5be61324b7eadc6e22871326461,Project Approval,MessageStatus.UNDELIVERED,2023-05-01 23:22:20,<<class 'syft.service.project.project.Project'...
1,syft.service.message.messages.Message,fcb593575fa340f4a65be8c4357015db,Approval Request,MessageStatus.UNDELIVERED,2023-05-01 23:21:05,<<class 'syft.service.request.request.Request'...


In [5]:
from syft import MessageStatus
messages = domain_client.api.services.messages.get_all_for_status(MessageStatus.UNDELIVERED)
messages

,type,id,subject,status,created_at,linked_obj
0,syft.service.message.messages.Message,8cf4d5be61324b7eadc6e22871326461,Project Approval,MessageStatus.UNDELIVERED,2023-05-01 23:22:20,<<class 'syft.service.project.project.Project'...
1,syft.service.message.messages.Message,fcb593575fa340f4a65be8c4357015db,Approval Request,MessageStatus.UNDELIVERED,2023-05-01 23:21:05,<<class 'syft.service.request.request.Request'...


In [6]:
assert len(messages) == 2

In [7]:
delivered_messages = domain_client.api.services.messages.get_all_for_status(status=sy.MessageStatus.DELIVERED)
delivered_messages

[]

In [8]:
assert len(delivered_messages) == 0

In [9]:
undelivered_messages = domain_client.api.services.messages.get_all_for_status(status=sy.MessageStatus.UNDELIVERED)
undelivered_messages

,type,id,subject,status,created_at,linked_obj
0,syft.service.message.messages.Message,8cf4d5be61324b7eadc6e22871326461,Project Approval,MessageStatus.UNDELIVERED,2023-05-01 23:22:20,<<class 'syft.service.project.project.Project'...
1,syft.service.message.messages.Message,fcb593575fa340f4a65be8c4357015db,Approval Request,MessageStatus.UNDELIVERED,2023-05-01 23:21:05,<<class 'syft.service.request.request.Request'...


In [10]:
assert len(undelivered_messages) == 2

In [11]:
project_message = None
for message in messages:
    if issubclass(message.linked_obj.object_type, sy.service.project.project.Project):
        project_message = message

In [12]:
assert project_message is not None

In [13]:
project = project_message.link
project

```python
class Project:
  id: str = 681d179281464bdfb8b92ca85d6c3a7e
  name: str = "My Cool UN Project"
  description: str = "Hi, I want to calculate the trade volume in million's with my cool code."
  user_verify_key: str = 0f6e3c166aca10b61249a83e066c3e78dec5562a32b558bf641e0393b696b0cd
  requests: str = [syft.service.request.request.Request]

```

In [14]:
assert len(project.requests) == 1

In [15]:
request = project.requests[0]
request

```python
class Request:
  id: str = 451b23f99cdc49c39531caf811282b2d
  requesting_user_verify_key: str = 0f6e3c166aca10b61249a83e066c3e78dec5562a32b558bf641e0393b696b0cd
  approving_user_verify_key: str = None
  request_time: str = 2023-05-01 23:22:20
  approval_time: str = None
  status: str = RequestStatus.PENDING
  node_uid: str = 7bca415d13ed1ec841f0d0aede098dbb
  request_hash: str = "0a3d37e60be39a57dc88040bc02fb243854b3b96acb857c5e8831411bda30164"
  changes: str = [syft.service.request.request.UserCodeStatusChange, syft.service.request.request.UserCodeStatusChange]

```

In [16]:
func = request.changes[0].link
func

```python
class UserCode:
  id: str = 389ebef717184d07955c932d6c5a9195
  node_uid: str = 7bca415d13ed1ec841f0d0aede098dbb
  user_verify_key: str = 0f6e3c166aca10b61249a83e066c3e78dec5562a32b558bf641e0393b696b0cd
  raw_code: str = "@sy.syft_function(input_policy=sy.ExactMatch(trade_data=mock),
                  output_policy=sy.SingleExecutionExactOutput())
def sum_trade_value_mil(trade_data):
    import pandas as pd
    from opendp.mod import enable_features
    enable_features('contrib')
    from opendp.measurements import make_base_laplace
    aggregate = 0.
    base_lap = make_base_laplace(scale=5.)
    noise = base_lap(aggregate)

    df = trade_data
    total = df["Trade Value (US$)"].sum()
    return (float(total / 1_000_000), float(noise))
"
  input_policy_type: str = <class 'syft.service.policy.policy.ExactMatch'>
  input_policy_init_kwargs: str = {NodeView(node_name='test-domain-1', verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f): {'trade_data': <UID: f3e8e3c990904166a6466687a09dfb19>}}
  input_policy_state: str = b''
  output_policy_type: str = <class 'syft.service.policy.policy.OutputPolicyExecuteOnce'>
  output_policy_init_kwargs: str = {}
  output_policy_state: str = b''
  parsed_code: str = "def user_func_sum_trade_value_mil_0f6e3c166aca10b61249a83e066c3e78dec5562a32b558bf641e0393b696b0cd_5cd11d8bc44ba9ca10a6a7f1ce43e49dfbe8f969c6370ef5d062a6390d12a4a0(trade_data):

    def sum_trade_value_mil(trade_data):
        import pandas as pd
        from opendp.mod import enable_features
        enable_features('contrib')
        from opendp.measurements import make_base_laplace
        aggregate = 0.0
        base_lap = make_base_laplace(scale=5.0)
        noise = base_lap(aggregate)
        df = trade_data
        total = df['Trade Value (US$)'].sum()
        return (float(total / 1000000), float(noise))
    result = sum_trade_value_mil(trade_data=trade_data)
    return result"
  service_func_name: str = "sum_trade_value_mil"
  unique_func_name: str = "user_func_sum_trade_value_mil_0f6e3c166aca10b61249a83e066c3e78dec5562a32b558bf641e0393b696b0cd_5cd11d8bc44ba9ca10a6a7f1ce43e49dfbe8f969c6370ef5d062a6390d12a4a0"
  user_unique_func_name: str = "user_func_sum_trade_value_mil_0f6e3c166aca10b61249a83e066c3e78dec5562a32b558bf641e0393b696b0cd"
  code_hash: str = "5cd11d8bc44ba9ca10a6a7f1ce43e49dfbe8f969c6370ef5d062a6390d12a4a0"
  signature: str = (trade_data)
  status: str = {NodeView(node_name='test-domain-1', verify_key=aec6ea4dfc049ceacaeeebc493167a88a200ddc367b1fa32da652444b635d21f): <UserCodeStatus.SUBMITTED: 'submitted'>}
  input_kwargs: str = ['trade_data']
  enclave_metadata: str = None

```

In [17]:
print(func.code)

@sy.syft_function(input_policy=sy.ExactMatch(trade_data=mock),
                  output_policy=sy.SingleExecutionExactOutput())
def sum_trade_value_mil(trade_data):
    import pandas as pd
    from opendp.mod import enable_features
    enable_features('contrib')
    from opendp.measurements import make_base_laplace
    aggregate = 0.
    base_lap = make_base_laplace(scale=5.)
    noise = base_lap(aggregate)

    df = trade_data
    total = df["Trade Value (US$)"].sum()
    return (float(total / 1_000_000), float(noise))



In [18]:
func.node_uid

<UID: 7bca415d13ed1ec841f0d0aede098dbb>

In [19]:
func.assets

,type,id
0,syft.service.dataset.dataset.Asset,05926243c7fb48e2b61ebc1973181d43


In [20]:
asset = func.assets[0]
asset

```python
Asset: canada_trade_flow
Pointer Id: f3e8e3c990904166a6466687a09dfb19
Description: all the datas
Total Data Subjects: 1
Shape: (10, 22)
Contributors: 1
	Andrew Trask: andrew@openmined.org

```

In [21]:
assert len(asset.data_subjects) == 1

In [22]:
mock = asset.mock
mock

,Classification,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter Code,Reporter,...,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit Code,Qty Unit,Qty,Netweight (kg),Trade Value (US$),Flag
10,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Bangladesh,NaN,19,"Preparations of cereals, flour, starch or milk...",0,NaN,NaN,0.0,227222,0
11,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Haiti,NaN,19,"Preparations of cereals, flour, starch or milk...",0,NaN,NaN,0.0,14748,0
12,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Guatemala,NaN,19,"Preparations of cereals, flour, starch or milk...",0,NaN,NaN,0.0,1314,0
13,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Iraq,NaN,19,"Preparations of cereals, flour, starch or milk...",0,NaN,NaN,0.0,1825,0
14,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Israel,NaN,19,"Preparations of cereals, flour, starch or milk...",0,NaN,NaN,0.0,1063627,0
15,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Italy,NaN,19,"Preparations of cereals, flour, starch or milk...",0,NaN,NaN,0.0,8359327,0
16,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Jordan,NaN,19,"Preparations of cereals, flour, starch or milk...",0,NaN,NaN,0.0,16858,0
17,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Rep. of Moldova,NaN,19,"Preparations of cereals, flour, starch or milk...",0,NaN,NaN,0.0,29897,0
18,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Nigeria,NaN,19,"Preparations of cereals, flour, starch or milk...",0,NaN,NaN,0.0,22235,0
19,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Slovenia,NaN,19,"Preparations of cereals, flour, starch or milk...",0,NaN,NaN,0.0,1328,0


In [23]:
assert mock.shape == (10, 22)

In [24]:
data = asset.data
data

,Classification,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter Code,Reporter,...,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit Code,Qty Unit,Qty,Netweight (kg),Trade Value (US$),Flag
0,HS,2021,202102,February 2021,4,0,1,Imports,124,Canada,...,"Other Asia, nes",NaN,6117,"Clothing accessories; made up, knitted or croc...",0,NaN,NaN,NaN,9285,0
1,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Egypt,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,116604,0
2,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,United Kingdom,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,1495175,0
3,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,United Rep. of Tanzania,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,2248,0
4,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Singapore,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,47840,0
5,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Viet Nam,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,3526,0
6,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,South Africa,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,5462,0
7,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Spain,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,311425,0
8,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Sweden,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,11786,0
9,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Venezuela,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,33715,0


In [25]:
assert data.shape == (10, 22)

In [26]:
op = func.output_policy_type
op

syft.service.policy.policy.OutputPolicyExecuteOnce

In [27]:
print(op.policy_code)

class Policy(SyftObject):
    # version
    __canonical_name__ = "Policy"
    __version__ = SYFT_OBJECT_VERSION_1

    id: UID
    init_kwargs: Dict[Any, Any] = {}

    def __init__(self, *args, **kwargs) -> None:
        if "init_kwargs" in kwargs:
            init_kwargs = kwargs["init_kwargs"]
            del kwargs["init_kwargs"]
        else:
            init_kwargs = deepcopy(kwargs)
            if "id" in init_kwargs:
                del init_kwargs["id"]
        super().__init__(init_kwargs=init_kwargs, *args, **kwargs)

    @classmethod
    @property
    def policy_code(cls) -> str:
        mro = reversed(cls.mro())
        op_code = ""
        for klass in mro:
            if "Policy" in klass.__name__:
                op_code += inspect.getsource(klass)
                op_code += "\n"
        return op_code

    def public_state() -> None:
        raise NotImplementedError

    @property
    def valid(self) -> Union[SyftSuccess, SyftError]:
        return SyftSuccess(message

In [28]:
print(func.raw_code)

@sy.syft_function(input_policy=sy.ExactMatch(trade_data=mock),
                  output_policy=sy.SingleExecutionExactOutput())
def sum_trade_value_mil(trade_data):
    import pandas as pd
    from opendp.mod import enable_features
    enable_features('contrib')
    from opendp.measurements import make_base_laplace
    aggregate = 0.
    base_lap = make_base_laplace(scale=5.)
    noise = base_lap(aggregate)

    df = trade_data
    total = df["Trade Value (US$)"].sum()
    return (float(total / 1_000_000), float(noise))



In [29]:
users_function = func.unsafe_function

In [30]:
real_result = users_function(trade_data=data)
real_result

(2.037066, 4.249859337131129)

In [31]:
result = request.accept_by_depositing_result(real_result)
result

<class 'syft.service.response.SyftSuccess'>: Request 451b23f99cdc49c39531caf811282b2d changes applied

In [32]:
assert isinstance(result, sy.SyftSuccess)

In [33]:
if node.node_type.value == "python":
    node.land()

Stopping test-domain-1
